In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors3D
import pandas as pd
from Hyprid import polFeature
import numpy as np
import sys
from rdkit.Chem import Descriptors

In [3]:
def hybConn(mol):
    sp3sp3 = 0
    sp3sp2 = 0
    sp3sp = 0
    sp2sp = 0
    sp2sp2 = 0
    spsp = 0
    sp3s = 0
    sp2s = 0
    sps = 0
    other = 0
    for bond in mol.GetBonds():
        atom1 = bond.GetBeginAtom()
        atom2 = bond.GetEndAtom()
        hyb1 = str(atom1.GetHybridization())
        hyb2 = str(atom2.GetHybridization())
        if   hyb1=="SP3" and hyb2=="SP3":sp3sp3+=1
        elif hyb1=="SP3" and hyb2=="SP2":sp3sp2+=1
        elif hyb1=="SP3" and hyb2=="SP" :sp3sp+=1
        elif hyb1=="SP3" and hyb2=="S"  :sp3s+=1
        elif hyb1=="S"   and hyb2=="SP3":sp3s+=1
        elif hyb1=="SP2" and hyb2=="SP3":sp3sp2+=1
        elif hyb1=="SP2" and hyb2=="SP2":sp2sp2+=1
        elif hyb1=="SP2" and hyb2=="SP" :sp2sp+=1
        elif hyb1=="SP2" and hyb2=="S"  :sp2s+=1
        elif hyb1=="S"   and hyb2=="SP2":sp2s+=1
        elif hyb1=="SP"  and hyb2=="SP3":sp3sp+=1
        elif hyb1=="SP"  and hyb2=="SP2":sp2sp+=1
        elif hyb1=="SP"  and hyb2=="SP" :spsp+=1
        elif hyb1=="SP"  and hyb2=="S"  :sps+=1
        elif hyb1=="S"   and hyb2=="SP" :sps+=1
        else:other+=1
    return [sp3sp3,sp3sp2,sp3sp,sp3s,sp2sp2,sp2sp,sp2s,spsp,sps,other]

In [4]:
def getMolDescriptors(mol):
    res = []
    for nm,fn in Descriptors._descList:
        if nm != 'Ipc':
            try:
                val = fn(mol)
                res.append(val)
            except:
                res.append(None)
                print(nm)
    return res

In [5]:
# Load and preprocess data
filename = "/bindingdb_may5th_g12c_169.tsv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename,sep='\t')
df.dropna(inplace=True)
hybd_names = ['FC','N','H','Cl','Br',
                 'I','P','F','Se','Si',
                 'S_sp3','S_sp2','S_sp',
                 'C_sp3','C_sp2','C_sp',
                 'N_sp3','N_sp2','N_sp',
                 'O_sp3','O_sp2','O_sp',
                 'I1','I2','I3']
bondhybd_names = ['sp3sp3','sp3sp2','sp3sp','sp3s',
                  'sp2sp2','sp2sp','sp2s','spsp','sps','other']
count = 0
dcr_names = [nm for nm,fn in Descriptors._descList]
dcr_names.remove('Ipc')
Cnames = ["ChEMBL ID","Smiles"]+hybd_names+bondhybd_names+dcr_names
print("Cnames", len(Cnames))
DF = pd.DataFrame(columns=Cnames)

Cnames 253


In [6]:
filename = "/bindingdb_may5th_g12c_169.tsv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename, sep='\t')
df.dropna(subset=['IC50 (nM)'], inplace=True)
df

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.1,UniProt (SwissProt) Entry Name of Target Chain.1,UniProt (SwissProt) Primary ID of Target Chain.1,UniProt (SwissProt) Secondary ID(s) of Target Chain.1,UniProt (SwissProt) Alternative ID(s) of Target Chain.1,UniProt (TrEMBL) Submitted Name of Target Chain.1,UniProt (TrEMBL) Entry Name of Target Chain.1,UniProt (TrEMBL) Primary ID of Target Chain.1,UniProt (TrEMBL) Secondary ID(s) of Target Chain.1,UniProt (TrEMBL) Alternative ID(s) of Target Chain.1
0,1419075,C[C@H](Oc1cc(nc(n1)-c1noc2c1CCC[C@@]21CCCc2sc(...,InChI=1S/C32H39FN8O2S/c1-18(22-13-19(33)16-40(...,UHFVWDBPFPIPSX-KVSFCKLASA-N,678321,"US20240174690, Example II-165::US20240174690, ...",GTPase KRas [G12D]/Son of sevenless homolog 1 ...,Homo sapiens,NaN,0.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
1,1419078,C[C@H](Oc1cc(nc(n1)-c1noc2c1CCC[C@@]21CCCc2sc(...,InChI=1S/C33H41FN8O2S/c1-19(23-14-20(34)17-40(...,RQPYOJQAMGQIAO-XCVAVQMESA-N,678305,"US20240174690, Example II-149::US20240174690, ...",GTPase KRas [G12D]/Son of sevenless homolog 1 ...,Homo sapiens,NaN,0.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
2,1418465,C[C@H](Oc1cc(nc(n1)-c1onc2c1CCC[C@@]21CCCc2sc(...,InChI=1S/C33H42N8O2S/c1-20(23-8-5-15-40(23)2)4...,JZPZUNIKIOONNK-INZDWPOASA-N,678102,"US20240174690, Example I-3",GTPase KRas [G12D]/Son of sevenless homolog 1 ...,Homo sapiens,NaN,1.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
3,1418468,C[C@H](Oc1cc(nc(n1)-c1onc2c1CCC[C@@]21CCCc2sc(...,InChI=1S/C32H40N8O3S/c1-19(22-7-5-12-39(22)2)4...,XDXFVTLDOBJHDT-ASMFUDSRSA-N,678112,"US20240174690, Example I-4",GTPase KRas [G12D]/Son of sevenless homolog 1 ...,Homo sapiens,NaN,1.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
4,1418473,C[C@H](Oc1cc(nc(n1)-c1onc2c1CCC[C@@]21CCCc2sc(...,InChI=1S/C32H40N8O2S/c1-19(22-7-5-14-39(22)2)4...,WHGPXTQJFBAXFS-ASMFUDSRSA-N,678125,"US20240174690, Example I-5",GTPase KRas [G12D]/Son of sevenless homolog 1 ...,Homo sapiens,NaN,1.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,1448763,CN(C)CC1(COc2nc3CN(Cc3c(n2)N2CC(O)CC(C2)NC(=O)...,"InChI=1S/C34H43IN6O6/c1-33(2,3)47-32(45)36-21-...",MNFLYWGEXBAVLL-UHFFFAOYSA-N,698857,"US20240317759, Example 67::BDBM698872::tert-bu...",version 2 g12d,NaN,NaN,467200.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
454,1448724,CN(C)CC1(COc2nc3CN(Cc3c(n2)N2CCCC3OCCC23)C(=O)...,InChI=1S/C31H36IN5O4/c1-35(2)17-31(9-10-31)18-...,VSVFZKVJLCXVCH-UHFFFAOYSA-N,698844,(2-((1- ((dimethylamino)methyl)cyclopropyl) me...,version 2 g12d,NaN,NaN,481800.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
455,1448604,CN(C)CC1(COc2nc3CN(Cc3c(n2)N2CCCC(CC2)C#N)C(=O...,InChI=1S/C31H35IN6O3/c1-36(2)18-31(9-10-31)19-...,FVNHHOXERXKJGA-UHFFFAOYSA-N,698817,1-(2-((1- ((dimethylamino)methyl)cyclopropyl) ...,version 2 g12d,NaN,NaN,533200.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
456,1448586,CC1CN(CCS1)c1nc(OCC2(CN(C)C)CC2)nc2CN(Cc12)C(=...,InChI=1S/C29H34IN5O3S/c1-18-13-34(9-10-39-18)2...,GMAJVTKHOUGEET-UHFFFAOYSA-N,698814,(2-((1- ((dimethylamino)methyl)cyclopropyl) me...,version 2 g12d,NaN,NaN,544600.0,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Iterate through ChEMBL IDs and SMILES
count = 0
for i, (chembl_id, smiles) in enumerate(zip(df["BindingDB Reactant_set_id"], df["Ligand SMILES"])):
    try:
        count += 1
        if count == 100:
            count = 0
            print("Processing molecule number", i)

        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"Skipping {chembl_id}: Invalid SMILES '{smiles}'")
            continue

        mol = Chem.AddHs(mol)  # Add hydrogens
        status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())

        if status == 0:
            t = getMolDescriptors(mol)
            hyp = polFeature(mol)
            bhyp = hybConn(mol)
            I = [Descriptors3D.PMI1(mol), Descriptors3D.PMI2(mol), Descriptors3D.PMI3(mol)]

            DF.loc[i] = [chembl_id, smiles] + hyp + bhyp + I + t
            print(f"Processed {chembl_id} successfully")
        else:
            print(f"Skipping {chembl_id}: 3D conformer generation failed")

    except Exception as e:
        print(f"Error processing {chembl_id} ({smiles}): {repr(e)}")

# Save to CSV
output_filename = filename.split(".")[0] + "g12c_Hyb_Features.csv"
DF.to_csv(output_filename, index=False)
print(f"Saved output to {output_filename}")

Processed 1419075 successfully
Processed 1419078 successfully
Processed 1418465 successfully
Processed 1418468 successfully
Processed 1418473 successfully
Processed 1418476 successfully
Processed 1418480 successfully
Processed 1418484 successfully
Processed 1418486 successfully
Processed 1418488 successfully
Processed 1418490 successfully
Processed 1418508 successfully
Processed 1418541 successfully
Processed 1418544 successfully
Processed 1418550 successfully
Processed 1418555 successfully
Processed 1418560 successfully
Processed 1418564 successfully
Processed 1418566 successfully
Processed 1418568 successfully
Processed 1418570 successfully
Processed 1418572 successfully
Processed 1418574 successfully
Processed 1418580 successfully
Processed 1418582 successfully
Processed 1418584 successfully
Processed 1418588 successfully
Processed 1418590 successfully
Processed 1418594 successfully
Processed 1418597 successfully
Processed 1418600 successfully
Processed 1418605 successfully
Processe

[15:14:42] Explicit valence for atom # 29 N, 4, is greater than permitted


Processed 1418536 successfully
Processed 1418652 successfully
Processed 1418771 successfully
Processing molecule number 199
Processed 1418806 successfully
Processed 1418825 successfully
Processed 1418827 successfully
Processed 1418831 successfully
Processed 1418848 successfully
Processed 1418872 successfully
Processed 1418883 successfully
Processed 1418914 successfully
Processed 1418515 successfully
Processed 1418779 successfully
Processed 1418793 successfully
Processed 1418839 successfully
Processed 1418525 successfully
Processed 1418534 successfully
Processed 1418548 successfully
Processed 1418760 successfully
Processed 1418781 successfully
Processed 1418787 successfully
Processed 1418829 successfully
Processed 1418891 successfully
Processed 1418906 successfully
Processed 1418910 successfully
Processed 1418959 successfully
Processed 1418511 successfully
Processed 1418732 successfully
Processed 1418777 successfully
Processed 1418894 successfully
Processed 1418647 successfully
Processe

In [9]:
import pandas as pd

# Load the files
df_ic50 = pd.read_csv("/bindingdb_may5th_g12c_169.tsv", sep='\t')  # Read TSV file
df_features = pd.read_csv("/bindingdb_may5th_g12c_169_Hyb_Features.csv")

# Merge on the respective ID columns
merged_df = df_features.merge(df_ic50[['BindingDB Reactant_set_id', 'IC50 (nM)']],
                              left_on="ChEMBL ID",
                              right_on="BindingDB Reactant_set_id",
                              how="left")

# Drop the duplicate ID column if needed
merged_df.drop(columns=["BindingDB Reactant_set_id"], inplace=True)

# Save the merged file
merged_df.to_csv("merged_features_IC50_g12c_169.csv", index=False)

print("Merge completed. File saved as new_merged_features_IC50_g12c.csv")

Merge completed. File saved as new_merged_features_IC50_g12c.csv
